In [3]:

# coding: utf-8

# In[4]:


from sklearn.model_selection import StratifiedKFold,KFold
from keras.preprocessing.text import one_hot
#from keras.preprocessing.text import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.callbacks import History
from numpy import zeros
from keras.layers.embeddings import Embedding
import pandas as pd
import numpy as np
from prf1 import precision, recall, f1_score
from keras.utils.vis_utils import plot_model
#import pydot
#import graphviz
from keras import backend as K
from keras.preprocessing import sequence
from keras.layers import LSTM
import numpy
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
#from confusionMetrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from prf1 import precision, recall, f1_score
from keras.utils import np_utils
from imblearn.over_sampling import SMOTE
#######Loading CSV file using Pandas
 
df=pd.read_excel('Airtel_MARKED.xlsx')


#print((df.info()))

df1=df['message']
label =df['Label']

#x_val, y_val = sm.fit_sample(x_val1, y_val1)
#print(label)
#Y=np_utils.to_categorical(Y_old)
#print(df1.info())
tk=Tokenizer()
tk.fit_on_texts(df1)
index=tk.word_index
#print(index)
x = tk.texts_to_sequences(df1)
#print (x)
seed = 7
numpy.random.seed(seed)
##encoded_doc = tk.texts_to_matrix(df, mode='count')
##print (encoded_doc)
##max_length=15
##padded_docs = sequence.pad_sequences(encoded_doc, maxlen=max_length, padding='pre')
##print (padded_docs)
vocab_size = len(index)


#print(vocab_size)
encoded_docs=[one_hot(d,vocab_size) for d in df1] 
#print (hello)
##############################Padding###############
max_length=50
padded_docs = sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print padded_docs
#####################Encoded label#######
encoder = LabelEncoder()
encoder.fit(label)
encoded_label = encoder.transform(label)
#print(encoded_label)
labels = np_utils.to_categorical(encoded_label)
#print(labels)
#print('Shape of data tensor:', x_train.shape)
#print('Shape of label tensor:', labels.shape)



#x_train1, x_val1, y_train1, y_val1 = check_X_y(padded_docs, labels, accept_sparse="csc", dtype=np.float32, multi_output=True)
#x_train1, x_val1, y_train1, y_val1=train_test_split(padded_docs,labels, test_size=0.25,random_state=42)
#print(x_train1.shape)
#print(x_val1.shape)
#print(y_train1.shape)
#print(y_val1.shape)
#sm = SMOTE(kind='regular')
#x_train, y_train = sm.fit_sample(x_train1,y_train1)
#x_val,y_val=sm.fit_sample(x_val1,y_val1)
 
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
 
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))
 
# In[10]:
embedding_matrix =zeros((vocab_size+ 1, 100))
for word, i in tk.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector   
print ("Embedding Matrix")
 
 
 
#############embedding layers#########################
 
history=History()
predictions_train=[]
predictions_test=[]
filter_sizes = [3,4,5]
fold_training=numpy.zeros(shape=75)
fold_test=numpy.zeros(shape=75)
##embedding_vecor_length =32
model = Sequential()
model.add(Embedding(vocab_size+1, 100,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False))
##model.add(Embedding(vocab_size,8, input_length=embedding_vecor_length))
kfold = KFold(n_splits=2, shuffle=True, random_state=7)
cvscores = []
ith=1
for train,test in kfold.split(padded_docs,labels):
    #print('Fold=',ith)
    #ith=ith+1
    #print(train,test)
   # X_train, X_test = padded_docs[train], padded_docs[test]
    #Y_train, Y_test = labels[train], labels[test]
    model.add(LSTM(100,return_sequences=True))
    model.add(LSTM(200,return_sequences=True))
    model.add(LSTM(200,return_sequences=True))
    model.add(LSTM(200,return_sequences=True,recurrent_dropout=0.2))
    model.add(LSTM(100))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='RMSprop',
                  metrics=['acc', precision, recall, f1_score])
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision,recall,f1_score])
    print(model.summary())
    history=model.fit(padded_docs[train], labels[train],validation_data=(padded_docs[test],labels[test]),epochs=1, batch_size=20,verbose=2, callbacks=[history])
    plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
    predictions = model.predict(padded_docs[test])
    b=np.zeros_like(predictions)
    b[np.arange(len(predictions)), predictions.argmax(1)]=1
    print( metrics.classification_report(labels[test],b))

print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
fold_training=numpy.divide(fold_training,20)
print(fold_training)
fold_test=numpy.divide(fold_test,20)
print(fold_test)
accuracy = model.evaluate(padded_docs[test],labels[test], verbose=0)
print('Accuracy: %f' % (accuracy*100))
    
    
                               
                               
    
'''history=History()
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.ylabel('precision')
plt.xlabel('epochs')
plt.legend(['train', 'test'])
plt.show()'''



Total 400000 word vectors in Glove 6B 100d.
Embedding Matrix


/home/phdstudents/anaconda2/lib/python2.7/site-packages/keras/layers/recurrent.py:2112: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           157500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 200)           240800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 200)           320800    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 200)           320800    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total para

/home/phdstudents/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/phdstudents/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


ValueError: Input 0 is incompatible with layer lstm_6: expected ndim=3, found ndim=2

In [4]:
predictions = model.predict(padded_docs[test])

In [5]:
predictions

array([[0.27416563, 0.19861417, 0.31205517, 0.21516505],
       [0.27511615, 0.19561127, 0.31741408, 0.21185851],
       [0.27222916, 0.18945298, 0.34318075, 0.19513711],
       [0.27442098, 0.20008937, 0.3101625 , 0.21532713],
       [0.2738794 , 0.19891837, 0.31136346, 0.21583876],
       [0.27372432, 0.19976644, 0.3106749 , 0.21583433],
       [0.27375105, 0.19970416, 0.3106949 , 0.21584989],
       [0.27633438, 0.19442032, 0.32019523, 0.20905004],
       [0.2737843 , 0.19965546, 0.31076202, 0.21579821],
       [0.27381232, 0.19967881, 0.31073168, 0.21577719],
       [0.27370515, 0.19982591, 0.31055412, 0.21591482],
       [0.2736112 , 0.19979489, 0.3104157 , 0.21617827],
       [0.273842  , 0.19937739, 0.3109517 , 0.21582891],
       [0.27367356, 0.19977087, 0.3106066 , 0.21594895],
       [0.27365065, 0.19961485, 0.3106595 , 0.216075  ],
       [0.2738125 , 0.19940849, 0.31108335, 0.21569566],
       [0.27383292, 0.19950876, 0.31093797, 0.21572036],
       [0.27375826, 0.19969845,